In [11]:
import os
import graphviz
from collections import defaultdict

# Function to read the NFA from input.txt file
def read_nfa(input_file):
    nfa = {
        'states': set(),
        'alphabet': set(),
        'transitions': defaultdict(dict),
        'start_state': None,
        'accept_states': set()
    }

    with open(input_file, 'r') as file:
        for line in file:
            line = line.strip()

            if line.startswith("States:"):
                # Extract states from the line and remove brackets
                nfa['states'] = set(line.split(":")[1].strip().strip('{}').split(", "))

            elif line.startswith("Alphabet:"):
                # Extract alphabet from the line and remove brackets
                nfa['alphabet'] = set(line.split(":")[1].strip().strip('{}').split(", "))

            elif line.startswith("Start State:"):
                # Extract start state
                nfa['start_state'] = line.split(":")[1].strip()

            elif line.startswith("Accept States:"):
                # Extract accept states and remove brackets
                nfa['accept_states'] = set(line.split(":")[1].strip().strip('{}').split(", "))

            elif line.startswith("Transitions:"):
                # Now start reading the transitions in the format 'state -> symbol -> next_states'
                for transition in file:
                    transition = transition.strip()
                    if "->" in transition:
                        state, symbol, next_states = transition.split("->")
                        state = state.strip()
                        symbol = symbol.strip()
                        next_states = next_states.strip().split(", ")

                        # Add transitions to the NFA transition dictionary
                        nfa['transitions'].setdefault(state, {}).setdefault(symbol, []).extend(next_states)
    
    return nfa


In [20]:
from collections import defaultdict, deque

# Convert NFA to DFA using Powerset Construction Algorithm
def convert_nfa_to_dfa(nfa):
    dfa = {
        'states': set(),
        'alphabet': nfa['alphabet'],
        'transitions': defaultdict(dict),
        'start_state': None,
        'accept_states': set()
    }

    # Start state of DFA is the epsilon closure of the NFA start state
    start_state = frozenset([nfa['start_state']])
    dfa['start_state'] = str(start_state)  # Convert to string for consistent use
    dfa['states'].add(str(start_state))

    # Queue to explore the DFA states
    queue = deque([start_state])
    visited = set([str(start_state)])

    while queue:
        current_dfa_state = queue.popleft()

        for symbol in nfa['alphabet']:
            next_nfa_states = set()
            # Calculate the set of NFA states reachable from current_dfa_state on this symbol
            for nfa_state in current_dfa_state:
                if symbol in nfa['transitions'].get(nfa_state, {}):
                    next_nfa_states.update(nfa['transitions'][nfa_state][symbol])

            # Create the corresponding DFA state
            next_dfa_state = frozenset(next_nfa_states)
            next_dfa_state_str = str(next_dfa_state)  # Convert to string

            if next_dfa_state:
                dfa['transitions'][str(current_dfa_state)][symbol] = next_dfa_state_str
                if next_dfa_state_str not in visited:
                    visited.add(next_dfa_state_str)
                    queue.append(next_dfa_state)
                    dfa['states'].add(next_dfa_state_str)

            # Check if the new DFA state is an accept state
            if any(state in nfa['accept_states'] for state in next_nfa_states):
                dfa['accept_states'].add(next_dfa_state_str)

    return dfa


def minimize_dfa(dfa):
    # Step 1: Initialize partitions
    P = [dfa['accept_states'], dfa['states'] - dfa['accept_states']]
    W = [dfa['accept_states'].copy()]  # States that need to be split

    # Step 2: Refining partitions
    while W:
        A = W.pop()

        for symbol in dfa['alphabet']:
            # Get all states that transition on 'symbol' to a state in 'A'
            X = {state for state in dfa['states'] if symbol in dfa['transitions'].get(state, {}) and dfa['transitions'][state][symbol] in A}

            for Y in P[:]:
                intersection = Y & X
                difference = Y - X

                if intersection and difference:
                    # Replace Y in P with two sets: intersection and difference
                    P.remove(Y)
                    P.append(intersection)
                    P.append(difference)

                    # Add the smaller of the two to W
                    if intersection in W:
                        W.append(difference)
                    else:
                        W.append(intersection)

    # Step 3: Create the minimized DFA
    minimized_dfa = {
        'states': set(),
        'alphabet': dfa['alphabet'],
        'transitions': defaultdict(dict),
        'start_state': None,
        'accept_states': set()
    }

    # Map old states to new states (partition groups)
    state_map = {}
    for partition in P:
        new_state = str(frozenset(partition))  # Convert to string
        minimized_dfa['states'].add(new_state)

        for state in partition:
            state_map[state] = new_state
            if state == dfa['start_state']:
                minimized_dfa['start_state'] = new_state
            if state in dfa['accept_states']:
                minimized_dfa['accept_states'].add(new_state)

    # Define transitions for minimized DFA
    for old_state, transitions in dfa['transitions'].items():
        new_state = state_map[old_state]
        for symbol, next_old_state in transitions.items():
            minimized_dfa['transitions'][new_state][symbol] = state_map[next_old_state]

    return minimized_dfa



In [18]:
import graphviz

def generate_graph(automaton, output_file):
    dot = graphviz.Digraph()
    
    # Add nodes
    for state in automaton.get('states', []):
        if state in automaton.get('accept_states', []):
            dot.node(state, shape='doublecircle')
        else:
            dot.node(state, shape='circle')
    
    # Create edges
    for state, transitions in automaton.get('transitions', {}).items():
        for symbol, next_states in transitions.items():
            for next_state in next_states:
                dot.edge(state, next_state, label=symbol)
    
    # Render the graph to a file
    dot.render(output_file, format='png', cleanup=True)


In [21]:
# Main function to process each test case
def process_testcase(testcase_dir):
    input_file = os.path.join(testcase_dir, 'input.txt')
    dfa_output_file = os.path.join(testcase_dir, 'dfa_image')
    minimized_dfa_output_file = os.path.join(testcase_dir, 'minimized_dfa_image')

    # Read NFA from input.txt
    nfa = read_nfa(input_file)
    
    # Convert NFA to DFA
    dfa = convert_nfa_to_dfa(nfa)
    
    # Generate DFA graph image
    generate_graph(dfa, dfa_output_file)

    # Minimize DFA
    minimized_dfa = minimize_dfa(dfa)
    
    # Generate minimized DFA graph image
    generate_graph(minimized_dfa, minimized_dfa_output_file)

# Loop through each test case folder and process
def process_all_testcases(testcases_root):
    for testcase in os.listdir(testcases_root):
        testcase_dir = os.path.join(testcases_root, testcase)
        if os.path.isdir(testcase_dir):
            print(f"Processing {testcase_dir}...")
            process_testcase(testcase_dir)

# Run the script for all test cases
if __name__ == "__main__":
    process_all_testcases('testcases')

Processing testcases/testcase-3...
Processing testcases/testcase-2...


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.967177 to fit


Processing testcases/testcase-1...
